In [1]:
%matplotlib inline

import IPython.display as ipd
import torch

import commons
import utils
from models import SynthesizerTrn
from text.symbols import symbols
from text.cleaners import japanese_phrase_cleaners
from text import cleaned_text_to_sequence
from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = cleaned_text_to_sequence(text)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [2]:
hps = utils.get_hparams_from_file("./configs/ljs_base.json")

In [ ]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model)
_ = net_g.eval()

_ = utils.load_checkpoint("./logs/ljs_base/G_63000.pth", net_g, None)

In [ ]:
def tts(text):
  stn_tst = get_text(japanese_phrase_cleaners(text), hps)
  with torch.no_grad():
    x_tst = stn_tst.unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1.1)[0][0,0].data.float().numpy()
  ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))

In [ ]:
baixue = "どうしてこうなるんだろう。初めて、好きな人が出来た。一生ものの友だちができた。嬉しいことが二つ重なって。その二つの嬉しさが、また、たくさんの嬉しさを連れてきてくれて。夢のように幸せな時間を手に入れたはずなのに…なのに、どうして、こうなっちゃうんだろう。"
japanese_phrase_cleaners(baixue)

In [ ]:
tts(baixue)

In [ ]:
tts("")
